### Beaches


In [1]:
pip install overpy


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install geopandas


Note: you may need to restart the kernel to use updated packages.


In [3]:
import overpy

api = overpy.Overpass()

# Query for beaches in Denmark using a more accurate bounding box
result = api.query("""
[out:json];
(
  node["natural"="beach"](54.5,8.0,57.9,15.2);
  way["natural"="beach"](54.5,8.0,57.9,15.2);
  relation["natural"="beach"](54.5,8.0,57.9,15.2);
);
(._;>;);
out body;
""")

# Structure the data
nodes = [{'name': beach.tags.get('name', 'n/a'), 'location': (beach.lon, beach.lat)} for beach in result.nodes]
ways = [{'name': way.tags.get('name', 'n/a'), 'nodes': [(node.lon, node.lat) for node in way.nodes]} for way in result.ways]

# For relations, fetch associated ways from the result
relations = []
for relation in result.relations:
    relation_ways = [way for way in result.ways if way.id in [member.ref for member in relation.members if member.role == "outer"]]
    relations.append({'name': relation.tags.get('name', 'n/a'), 'ways': relation_ways})


In [4]:
import csv

def get_centroid(nodes):
    """Calculate the centroid of a list of nodes."""
    x_coords = [node[0] for node in nodes]
    y_coords = [node[1] for node in nodes]
    centroid_x = sum(x_coords) / len(nodes)
    centroid_y = sum(y_coords) / len(nodes)
    return (centroid_x, centroid_y)

beaches_centroids = {}

# Handle nodes:
for node in nodes:
    beach_name = node['name']
    if beach_name not in beaches_centroids:
        beaches_centroids[beach_name] = node['location']

# Handle ways:
for way in ways:
    beach_name = way['name']
    if beach_name not in beaches_centroids:
        beaches_centroids[beach_name] = get_centroid(way['nodes'])

# Handle relations:
for relation in relations:
    beach_name = relation['name']
    if beach_name not in beaches_centroids:
        # For each way in the relation, retrieve its nodes and calculate its centroid
        relation_centroids = [get_centroid([(node.lon, node.lat) for node in way.nodes]) for way in relation['ways']]
        beaches_centroids[beach_name] = get_centroid(relation_centroids)

# Save to CSV
with open('Beaches.csv', 'w', newline='') as csvfile:
    fieldnames = ['Beach Name', 'Longitude', 'Latitude']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for name, location in beaches_centroids.items():
        writer.writerow({'Beach Name': name, 'Longitude': location[0], 'Latitude': location[1]})


In [5]:
import csv

# Combine all data
all_centroids = {
    "Beaches": beaches_centroids,
}

# Write data to CSV with UTF-8 encoding
with open("beaches.csv", "w", newline='', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file)
    # Write header
    writer.writerow(["Type", "Name", "Longitude", "Latitude"])
    
    for category, centroids in all_centroids.items():
        for name, location in centroids.items():
            if location != "No Centroid":
                writer.writerow([category, name, location[0], location[1]])
            else:
                writer.writerow([category, name, "No Centroid", "No Centroid"])

print("Data written to 'beaches.csv'")

Data written to 'beaches.csv'


In [6]:
import pandas as pd

# Assuming you've loaded your data into a DataFrame named 'df'
df = pd.read_csv('Beaches.csv')

# Reordering the columns
df = df[['Type', 'Name', 'Latitude', 'Longitude']]

# Saving it back to the CSV (or to a new CSV if you prefer)
df.to_csv('Beaches_right_order.csv', index=False)

### Forsøg 1

In [7]:
import folium

# Create a base map
m = folium.Map(location=[56.2, 11.6], zoom_start=6)  # centering the map around the middle of Denmark

# Define the bounding box coordinates for Denmark
bbox = [(54.5, 8.0), (54.5, 15.2), (57.9, 15.2), (57.9, 8.0), (54.5, 8.0)]

# Add bounding box to the map using PolyLine
folium.PolyLine(bbox, color="blue", weight=2.5, opacity=1).add_to(m)

# Show the map
m


In [8]:
import geopandas as gpd
import pandas as pd

# 1. Load the Denmark boundary shapefile
denmark_boundary = gpd.read_file("Danmark Shapefile/DNK_adm0.shp")

# 2. Load the CSV data for the beaches
beaches_df = pd.read_csv('Beaches_right_order.csv')

# Convert the CSV dataframe to a geodataframe
gdf_beaches = gpd.GeoDataFrame(beaches_df, 
                               geometry=gpd.points_from_xy(beaches_df.Longitude, beaches_df.Latitude))

# 3. Check if each beach is within Denmark's boundary
# The resulting object is a Boolean Series
in_denmark = gdf_beaches.geometry.within(denmark_boundary.at[0, 'geometry'])

# 4. Filter the beaches that are inside Denmark's boundary
beaches_inside_denmark = gdf_beaches[in_denmark]

# 5. Save the filtered data back to a new CSV
beaches_inside_denmark.drop(columns='geometry').to_csv('beaches_inside_denmark.csv', index=False)

print(f"{len(beaches_df) - len(beaches_inside_denmark)} beaches were outside Denmark and have been removed.")


215 beaches were outside Denmark and have been removed.
